# Importando bibliotecas que serão utilizadas no código

In [ ]:
# IMPORTAR BIBLIOTECAS NECESSÁRIAS

!pip install oracledb
import oracledb

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from IPython.display import display

----
----

### Criando conexão com o banco de dados oracle, utilizando a biblioteca

```
!pip install oracledb
import oracledb
```



In [ ]:
# CONFIGURAÇÃO DE CONEXÃO COM O BANCO DE DADOS ORACLE
connection = oracledb.connect(user='rm551451', password='fiap23', dsn='oracle.fiap.com.br:1521/orcl')

----
----

### Puxar os dados das tabelas, no banco de dados oracle e depois exibir os dados no console

> IA_PRODUTO

> IA_CLIENTE

> IA_HISTORICO_PESQUISA





In [ ]:
# Função para carregar os dados de produtos e características dos usuários
def load_data():
    query_produtos = "SELECT * FROM IA_PRODUTO"
    query_clientes = "SELECT * FROM IA_CLIENTE"
    query_historico = "SELECT * FROM IA_HISTORICO_PESQUISA"

    df_produtos = pd.read_sql(query_produtos, con=connection)
    df_clientes = pd.read_sql(query_clientes, con=connection)
    df_historico = pd.read_sql(query_historico, con=connection)

    return df_produtos, df_clientes, df_historico

# Acionando a função
df_produtos, df_clientes, df_historico = load_data()

# Exibindo os dados
display(df_produtos.head())
display(df_clientes.head())
display(df_historico.head())

----
----
### Processar os dados obtidos nos SELECTS

In [ ]:
# PRÉ-PROCESSAMENTO DOS DADOS
def preprocess_data(df_produtos, df_clientes, df_historico):
    # Merge dos dados relevantes
    df = pd.merge(df_historico, df_clientes, left_on='CLIENTE_ID', right_on='ID')
    df = pd.merge(df, df_produtos, left_on='PRODUTO_ID', right_on='ID')

    # Selecionar apenas as colunas relevantes para a recomendação
    features = ['COR_PELE', 'ESTADO_CIVIL', 'MARCA_PREFERIDA', 'NOME_PRODUTO']
    df = df[features]

    # Convertendo colunas categóricas em variáveis dummy
    df = pd.get_dummies(df, columns=['COR_PELE', 'ESTADO_CIVIL', 'MARCA_PREFERIDA'])

    return df

# ACIONANDO A FUNÇÃO DE PROCESSAMENTO DOS DADOS
df = preprocess_data(df_produtos, df_clientes, df_historico)

# Verificando o DataFrame resultante
display(df.head())



----
----

### Fazendo divisao entre dados de TREINO e dados de TESTE, para realizar o modelo de treinamento.

In [ ]:
# DIVIDINDO OS DADOS EM TREINO E TESTE
X = df.drop('NOME_PRODUTO', axis=1)
y = df['NOME_PRODUTO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TREINANDO UM MODELO DE CLASSIFICAÇÃO
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)



----
----

### Buscando dados do usuário no banco e enviando para a função de recomendação de produtos

### Função para recomendar o produto, de acordo com a analise e o treinamento do modelo

In [ ]:
# FUNÇÃO PARA RECOMENDAR PRODUTOS
def recommend_product(client_data):
    client_df = pd.DataFrame([client_data])
    client_df = pd.get_dummies(client_df)

    # Alinhar as colunas do client_df com as colunas usadas no treinamento
    client_df = client_df.reindex(columns=X.columns, fill_value=0)

    recommended_product = model.predict(client_df)
    return recommended_product[0]

# Buscar dados de um cliente específico no banco de dados
def get_client_data(client_id):
    query_cliente = f"SELECT * FROM IA_CLIENTE WHERE ID = {client_id}"
    df_cliente = pd.read_sql(query_cliente, con=connection)

    if df_cliente.empty:
        raise ValueError("Cliente não encontrado.")

    # Mapeando os dados do cliente para o formato esperado pela função recommend_product
    client_data = {
        'COR_PELE': df_cliente.iloc[0]['COR_PELE'],
        'ESTADO_CIVIL': df_cliente.iloc[0]['ESTADO_CIVIL'],
        'MARCA_PREFERIDA': df_cliente.iloc[0]['MARCA_PREFERIDA']
    }

    return client_data

# Exemplo de uso
try:
    client_id = 1  # Substitua pelo ID do cliente que você deseja buscar
    client_data = get_client_data(client_id)
    recommended_product = recommend_product(client_data)
    print(f"\n\nProduto recomendado: {recommended_product}\n\n")
except ValueError as e:
    print(e)


<ipython-input-40-c5ccedb69ab9>:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cliente = pd.read_sql(query_cliente, con=connection)
